# Домашнее задание

1. Обучить любую модель классификации на датасете IRIS до применения самописного PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке.


2. *Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции [numpy.linalg.svd()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html)

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

In [2]:
# Загрузим игрушечный датасет из sklearn
iris = datasets.load_iris()
X = iris.data
y = iris.target
X.shape

(150, 4)

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

lr = LinearRegression()

lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

R2_before_pca = r2_score(y_test, y_pred)
R2_before_pca

0.9442318571467433

In [4]:
def standard_scale(x):
    res = (x - x.mean(axis=0)) / x.std(axis=0)
    return res

In [5]:
X = X.astype(float)

X = standard_scale(X)

In [6]:
# Найдем собственные векторы и собственные значения
 
covariance_matrix = X.T @ X

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

print('Собственные значения и собственные векторы в порядке убывания:')
for i in eig_pairs:
    print(i)

Собственные значения и собственные векторы в порядке убывания:
(437.77467247979905, array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654]))
(137.10457072021043, array([-0.37741762, -0.92329566, -0.02449161, -0.06694199]))
(22.013531335697206, array([-0.71956635,  0.24438178,  0.14212637,  0.63427274]))
(3.1072254642928407, array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))


In [7]:
eig_sum = sum(eig_values)
var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описываемая каждой из компонент \n{var_exp}')

# а теперь оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описываемая каждой из компонент 
[72.96244541329992, 22.85076178670176, 3.668921889282871, 0.5178709107154739]
Кумулятивная доля дисперсии по компонентам 
[ 72.96244541  95.8132072   99.48212909 100.        ]


In [8]:
# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack([eig_pairs[i][1].reshape(4,1) for i in range(2)])

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.52106591 -0.37741762]
 [-0.26934744 -0.92329566]
 [ 0.5804131  -0.02449161]
 [ 0.56485654 -0.06694199]]


In [9]:
# Сформируем новую матрицу "объекты-признаки"
Z = X.dot(W)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(Z,y, test_size=0.3, random_state=42)

lr_pca = LinearRegression()

lr_pca.fit(X_train, y_train)
y_pred = lr_pca.predict(X_test)

R2_after_pca = r2_score(y_test, y_pred)
R2_before_pca,R2_after_pca

(0.9442318571467433, 0.906077721125034)

In [11]:
U, D, Vt = np.linalg.svd(X, full_matrices=False, compute_uv=True)

In [17]:
# Z_svd = np.dot(U, np.diag(D))
S_r = X.T @ X
W = Vt.T @ np.diag(D)
Z_svd = X @ W

In [18]:
X_train, X_test, y_train, y_test = train_test_split(Z_svd,y, test_size=0.3, random_state=42)

lr_svd = LinearRegression()

lr_svd.fit(X_train, y_train)
y_pred = lr_svd.predict(X_test)


R2_svd = r2_score(y_test, y_pred)

In [21]:
R2_before_pca,R2_after_pca,R2_svd

(0.9442318571467433, 0.906077721125034, 0.9442318571467434)